In [15]:
import cv2
from ultralytics import YOLO
from datetime import datetime
import os
import time
import base64
import psycopg2

# Load YOLO model (large version)
model = YOLO("yolov8l.pt")

# Open webcam
cap = cv2.VideoCapture(0)
person_warnings = 0
phone_warnings = 0
max_warnings = 3

last_person_warning_time = 0
last_phone_warning_time = 0
cooldown = 1.0

# Snapshot folder
os.makedirs("snapshots", exist_ok=True)

conn = psycopg2.connect(
   # paste your database credentials
)

cur = conn.cursor()

cur.execute(
    """CREATE TABLE IF NOT EXISTS Fair_Exam(
    id SERIAL PRIMARY KEY,
    student_name TEXT,
    name TEXT NOT NULL,
    ts TIMESTAMP NOT NULL,
    photo_b64 TEXT NOT NULL);
    """
)

conn.commit()

def save_snapshot(frame, name: str, student_name: str):
    ok, jpeg_bytes = cv2.imencode(".jpg", frame)
    if not ok:
        print("Couldn't encode snapshot")
        return None

    b64_str = base64.b64encode(jpeg_bytes.tobytes()).decode("utf-8")
    ts =datetime.now()

    cur.execute(
        "INSERT INTO Fair_Exam(student_name, name, ts, photo_b64) VALUES (%s, %s, %s, %s) RETURNING id",
        (student_name, name, ts, b64_str)
    )

    row_id = cur.fetchone()[0]
    conn.commit()
    print(f"Snapshot saved (id={row_id}, event = {name}, student = {student_name})")
    return row_id

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    # Run YOLO detection
    results = model(frame, conf=0.7, iou=0.5)

    people_cnt = 0
    phone_cnt = 0

    for box in results[0].boxes:
        cls_id = int(box.cls[0])
        X1, y1, X2, y2 = box.xyxy[0].int().tolist()
        conf = float(box.conf[0])

        if cls_id == 0:  # person
            people_cnt += 1
            color, label = (0, 255, 0), f"Person {conf:.2f}"
        elif cls_id == 67:  # cell phone
            phone_cnt += 1
            color, label = (0, 0, 255), f"Phone {conf:.2f}"
        else:
            continue  # ignore all other classes

        # Draw only person & phone
        cv2.rectangle(frame, (X1, y1), (X2, y2), color, 2)
        cv2.putText(frame, label, (X1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)

    # Add info text
    cv2.putText(frame, f"People: {people_cnt} | Phones: {phone_cnt}",
                (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)

    # Timestamp
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    now = time.time()

    # Multiple people warning
    if people_cnt >= 2 and (now - last_person_warning_time) >= cooldown:
        person_warnings += 1
        last_person_warning_time = now
        cv2.putText(frame, f" Multiple people detected! {person_warnings}/{max_warnings})", (20, 80),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 3)
        snap_path = f"snapshots/multiple_people_{timestamp}.jpg"
        cv2.imwrite(snap_path, frame)
        print(f"Snapshot saved: {snap_path}")
        save_snapshot(frame, "person_detected", student_name="Student1")

    # Phone warning
    if phone_cnt >= 1 and (now - last_phone_warning_time) >= cooldown:
        phone_warnings += 1
        last_phone_warning_time = now
        cv2.putText(frame, f" Phone detected! ({phone_warnings}/{max_warnings})", (20, 120),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 3)
        snap_path = f"snapshots/phone_{timestamp}.jpg"
        cv2.imwrite(snap_path, frame)
        print(f"Snapshot saved: {snap_path}")
        save_snapshot(frame, "phone_detected", student_name="Student1")

    # Auto logout
    if person_warnings >= max_warnings:
        print("Logged Out")
        break

    if phone_warnings >= max_warnings:
        print("Logged Out")
        break
    
    # Show output (display frame, not annotated_frame)
    cv2.imshow("Exam Monitoring", frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()



0: 480x640 2 persons, 1 tie, 1 chair, 763.6ms
Speed: 3.9ms preprocess, 763.6ms inference, 1.9ms postprocess per image at shape (1, 3, 480, 640)
Snapshot saved: snapshots/multiple_people_20250917_150838.jpg
Snapshot saved (id=4, event = person_detected, student = Student1)

0: 480x640 2 persons, 1 tie, 1 chair, 816.3ms
Speed: 7.0ms preprocess, 816.3ms inference, 2.1ms postprocess per image at shape (1, 3, 480, 640)
Snapshot saved: snapshots/multiple_people_20250917_150839.jpg
Snapshot saved (id=5, event = person_detected, student = Student1)

0: 480x640 2 persons, 1 tie, 1 chair, 606.0ms
Speed: 2.7ms preprocess, 606.0ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 persons, 1 tie, 530.3ms
Speed: 2.8ms preprocess, 530.3ms inference, 1.3ms postprocess per image at shape (1, 3, 480, 640)
Snapshot saved: snapshots/multiple_people_20250917_150841.jpg
Snapshot saved (id=6, event = person_detected, student = Student1)
Logged Out
